#### Generate Images

##### 1. Import and Load Data

In [1]:
# Install libraries first: geopandas, rasterio, shapely

import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely import box
from shapely.geometry import mapping

In [2]:
streets_fp = "nyc_files/nyc_streets.shp"
raster_fp = "nyc_files/nyc_raster.tif"

streets = gpd.read_file(streets_fp)
src = rasterio.open(raster_fp)

clean_raster_out_fp = "files/clean_raster.tif" # CHANGE THIS IF NEEDED. If using directory (e.g. /files), make sure it exists
street_images_out_fp = "files/streets" # CHANGE THIS IF NEEDED. If using directory (e.g. /files), make sure it exists

##### 2. Clip streets data to area of interest

In [3]:
# Unify CRS first

assert src.crs.is_projected
streets = streets.to_crs(32618)
assert streets.crs == src.crs

In [4]:
# get street bounds
src_bounds = src.bounds

# clip area
raster_extent_geom = box(
    src_bounds.left,
    src_bounds.bottom,
    src_bounds.right,
    src_bounds.top
)

# a dataframe with a single row and geometry, representing the raster bounds
raster_extent_gdf = gpd.GeoDataFrame(
    geometry=[raster_extent_geom],
    crs=src.crs
)

# clip!
streets_clipped = gpd.clip(streets, raster_extent_gdf)

##### 3. Remove any data not within radius 
This code block generates a raster image with street data + buffer

In [6]:
# Export all streets into a single image

# buffer 10m
streets_clipped_buffered = streets_clipped[:]
streets_clipped_buffered.geometry = streets_clipped_buffered.buffer(10)

buffer_geom = streets_clipped_buffered.union_all()

clipped_img, clipped_transform = mask(
        src,
        [mapping(buffer_geom)],
        crop=True
    )

clipped_meta = src.meta.copy()

clipped_meta.update({
    "height": clipped_img.shape[1],
    "width": clipped_img.shape[2],
    "transform": clipped_transform
})

with rasterio.open(clean_raster_out_fp, "w", **clipped_meta) as dst:
    dst.write(clipped_img)

##### 4. Export each street into its own image

In [ ]:
# Export each street into a separate image

streets_clipped_buffered[-2:-1].explore()

def generate_image(name, geometry, raster_src):
    print("CRS Equal? ", streets_clipped_buffered.crs == src.crs)
    
    clipped_img, clipped_transform = mask(
        raster_src,
        [mapping(geometry)],
        crop=True
    )

    clipped_meta = raster_src.meta.copy()
    clipped_meta.update({
        "height": clipped_img.shape[1],
        "width": clipped_img.shape[2],
        "transform": clipped_transform        
    })

    with rasterio.open(f"{street_images_out_fp}/{name}.tif", "w", **clipped_meta) as dst:
        dst.write(clipped_img)

for idx, street in streets_clipped_buffered.iterrows():
    generate_image(street["NAME"], street["geometry"], src)
    print(f"exported for street {idx}")


CRS Equal?  True
exported for street 1451
CRS Equal?  True
exported for street 1708
CRS Equal?  True
exported for street 1962
CRS Equal?  True
exported for street 1625
CRS Equal?  True
exported for street 1921
CRS Equal?  True
exported for street 1490
CRS Equal?  True
exported for street 1997
CRS Equal?  True
exported for street 1786
CRS Equal?  True
exported for street 1925
CRS Equal?  True
exported for street 1978
CRS Equal?  True
